In [ ]:
# Configuring an ARIMA Model
# The classical approach for fitting an ARIMA model is to follow the Box-Jenkins Methodology.
# This is a process that uses time series analysis and diagnostics to discover good parameters 
# for the ARIMA model.

# In summary, the steps of this process are as follows:

# Model Identification. Use plots and summary statistics to identify trends, seasonality, 
# and autoregression elements to get an idea of the amount of differencing and the size of the lag 
# that will be required.

# Parameter Estimation. Use a fitting procedure to find the coefficients of the regression model.

# Model Checking. Use plots and statistical tests of the residual errors to determine the amount 
# and type of temporal structure not captured by the model.
# The process is repeated until either a desirable level of fit is achieved on the in-sample 
# or out-of-sample observations (e.g. training or test datasets).

In [ ]:
# The ARIMA model for time series analysis and forecasting can be tricky to configure.

# There are 3 parameters that require estimation by iterative trial 
# and error from reviewing diagnostic plots and using 40-year-old heuristic rules.

# We can automate the process of evaluating a large number of hyperparameters 
# for the ARIMA model by using a grid search procedure.

In [ ]:
# Grid Searching Method

# Diagnostic plots of the time series can be used along with heuristic rules 
# to determine the hyperparameters of the ARIMA model.

# These are good in most, but perhaps not all, situations.

# We can automate the process of training and evaluating ARIMA models 
# on different combinations of model hyperparameters. 
# In machine learning this is called a grid search or model tuning.

In [ ]:
# In this tutorial, we will develop a method to grid search ARIMA hyperparameters for a one-step rolling forecast.

# The approach is broken down into two parts:

#    Evaluate an ARIMA model.
#    Evaluate sets of ARIMA parameters.

###### 1. Evaluate ARIMA Model

In [ ]:
# The dataset is split in two: 66% for the initial training dataset and the remaining 34% for the test dataset.

# Each time step of the test set is iterated. Just one iteration provides a model 
# that you could use to make predictions on new data. 
# The iterative approach allows a new ARIMA model to be trained each time step.

# A prediction is made each iteration and stored in a list. 
# This is so that at the end of the test set, all predictions can be compared to the list of expected values 
# and an error score calculated. In this case, a mean squared error score is calculated and returned.

In [9]:
import warnings
from pandas import read_csv
from datetime import datetime
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
warnings.filterwarnings("ignore")

In [10]:
# evaluate an ARIMA model for a given order (p,d,q)
def evaluate_arima_model(X, arima_order):
	# prepare training dataset
	train_size = int(len(X) * 0.66)
	train, test = X[0:train_size], X[train_size:]
	history = [x for x in train]
	# make predictions
	predictions = list()
	for t in range(len(test)):
		model = ARIMA(history, order=arima_order)
		model_fit = model.fit(disp=0)
		yhat = model_fit.forecast()[0]
		predictions.append(yhat)
		history.append(test[t])
	# calculate out of sample error
	error = mean_squared_error(test, predictions)
	return error

###### 2. Iterate ARIMA Parameters

In [ ]:
# Evaluating a suite of parameters is relatively straightforward.

# The user must specify a grid of p, d, and q ARIMA parameters to iterate. 
# A model is created for each parameter and its performance evaluated by calling the evaluate_arima_model() function 
# described in the previous section.

# The function must keep track of the lowest error score observed and the configuration that caused it. 
# This can be summarized at the end of the function with a print to standard out.

# We can implement this function called evaluate_models() as a series of four loops.

# There are two additional considerations. The first is to ensure the input data are floating point values 
# (as opposed to integers or strings), as this can cause the ARIMA procedure to fail.

# Second, the statsmodels ARIMA procedure internally uses numerical optimization procedures to find a set of coefficients for the model. 
# These procedures can fail, which in turn can throw an exception. We must catch these exceptions and skip those configurations 
# that cause a problem. This happens more often then you would think.

# Additionally, it is recommended that warnings be ignored for this code to avoid a lot of noise from running the procedure. 
# This can be done as follows:

In [11]:
# evaluate combinations of p, d and q values for an ARIMA model
def evaluate_models(dataset, p_values, d_values, q_values):
	dataset = dataset.astype('float32')
	best_score, best_cfg = float("inf"), None
	for p in p_values:
		for d in d_values:
			for q in q_values:
				order = (p,d,q)
				try:
					mse = evaluate_arima_model(dataset, order)
					if mse < best_score:
						best_score, best_cfg = mse, order
					print('ARIMA%s MSE=%.3f' % (order,mse))
				except:
					continue
	print('Best ARIMA%s MSE=%.3f' % (best_cfg, best_score))

In [ ]:
# Now that we have a procedure to grid search ARIMA hyperparameters, let’s test the procedure on two univariate time series problems.

###### Shampoo Sales Case Study

In [ ]:
# The monthly number of sales of shampoo over a 3-year period. The units are a sales count and there are 36 observations.

In [12]:
# load dataset
def parser(x):
	return datetime.strptime('190'+x, '%Y-%m')
series = read_csv('shampoo.csv', header=0, parse_dates=[0], index_col=0, squeeze=True, date_parser=parser)

In [ ]:
# Once loaded, we can specify a site of p, d, and q values to search 
# and pass them to the evaluate_models() function.

# We will try a suite of lag values (p) and just a few difference iterations (d) 
# and residual error lag values (q).

In [ ]:
# Putting this all together with the generic procedures defined in the previous section, 
# we can grid search ARIMA hyperparameters in the Shampoo Sales dataset.

In [7]:
# evaluate parameters
p_values = [0, 1, 2, 4, 6, 8, 10]
d_values = range(0, 3)
q_values = range(0, 3)
warnings.filterwarnings("ignore")
evaluate_models(series.values, p_values, d_values, q_values)

ARIMA(0, 0, 0) MSE=52425.268
ARIMA(0, 0, 1) MSE=38145.195
ARIMA(0, 0, 2) MSE=23989.599
ARIMA(0, 1, 0) MSE=18003.173
ARIMA(0, 1, 1) MSE=9558.316
ARIMA(0, 1, 2) MSE=6306.301
ARIMA(0, 2, 0) MSE=67339.808
ARIMA(0, 2, 1) MSE=18322.407
ARIMA(1, 0, 0) MSE=23113.098
ARIMA(1, 0, 2) MSE=7290.639
ARIMA(1, 1, 0) MSE=7121.369
ARIMA(1, 1, 1) MSE=7003.683
ARIMA(1, 2, 0) MSE=18608.012
ARIMA(2, 0, 0) MSE=10202.713
ARIMA(2, 1, 0) MSE=5689.927
ARIMA(2, 1, 1) MSE=7759.708
ARIMA(2, 2, 0) MSE=9860.949
ARIMA(4, 0, 0) MSE=10037.070
ARIMA(4, 1, 0) MSE=6649.593
ARIMA(4, 1, 1) MSE=6796.287
ARIMA(4, 2, 0) MSE=7596.327
ARIMA(4, 2, 1) MSE=4694.875
ARIMA(6, 1, 0) MSE=6810.076
ARIMA(6, 1, 1) MSE=4343.822
ARIMA(6, 2, 0) MSE=6261.124
ARIMA(8, 1, 0) MSE=6579.719
ARIMA(10, 1, 0) MSE=7532.391
Best ARIMA(6, 1, 1) MSE=4343.822


In [ ]:
# The best parameters of ARIMA(6, 1, 1) are reported at the end of the run with a mean squared error of 4,343.822.

###### Daily Female Births Case Study

In [ ]:
# The Daily Female Births dataset describes the number of daily female births in California in 1959.
# The units are a count and there are 365 observations. 

In [13]:
series = read_csv('daily-total-female-births.csv', header=0, index_col=0)

In [ ]:
# evaluate parameters
p_values = [0, 1, 2, 4, 6, 8, 10]
d_values = range(0, 3)
q_values = range(0, 3)
warnings.filterwarnings("ignore")
evaluate_models(series.values, p_values, d_values, q_values)

ARIMA(0, 0, 0) MSE=67.063
ARIMA(0, 0, 1) MSE=62.165
ARIMA(0, 0, 2) MSE=60.386
ARIMA(0, 1, 0) MSE=84.038
ARIMA(0, 1, 1) MSE=56.653
ARIMA(0, 1, 2) MSE=55.271
ARIMA(0, 2, 0) MSE=246.414
ARIMA(0, 2, 1) MSE=84.659
ARIMA(0, 2, 2) MSE=57.245
ARIMA(1, 0, 0) MSE=60.876
ARIMA(1, 0, 1) MSE=57.058
ARIMA(1, 1, 0) MSE=65.928
ARIMA(1, 1, 1) MSE=55.129
ARIMA(1, 1, 2) MSE=55.192
ARIMA(1, 2, 0) MSE=143.755
ARIMA(2, 0, 0) MSE=59.251
ARIMA(2, 0, 1) MSE=55.075
ARIMA(2, 0, 2) MSE=55.101
ARIMA(2, 1, 0) MSE=59.487
ARIMA(2, 1, 1) MSE=55.012
ARIMA(2, 1, 2) MSE=55.215
ARIMA(2, 2, 0) MSE=107.600
ARIMA(4, 0, 0) MSE=59.189
ARIMA(4, 0, 1) MSE=61.287
ARIMA(4, 0, 2) MSE=55.618
ARIMA(4, 1, 0) MSE=57.428
ARIMA(4, 1, 1) MSE=55.862
ARIMA(4, 1, 2) MSE=55.570
ARIMA(4, 2, 0) MSE=80.207
ARIMA(6, 0, 0) MSE=58.773
ARIMA(6, 0, 1) MSE=59.455
ARIMA(6, 1, 0) MSE=53.187
ARIMA(6, 1, 1) MSE=57.287
ARIMA(6, 1, 2) MSE=55.675
ARIMA(6, 2, 0) MSE=69.753
ARIMA(8, 0, 0) MSE=56.984
ARIMA(8, 0, 1) MSE=57.224
ARIMA(8, 0, 2) MSE=58.135
ARIMA(8, 

In [ ]:
# Running the example prints the ARIMA parameters and mean squared error for each configuration successfully evaluated.
# The best mean parameters are reported as ARIMA(6, 1, 0) with a mean squared error of 53.187.